
<img src="mc.png" style="float: left;" width="300"/>

# A Notebook for Processing GameTime Facebook Leads

You can use this page to lookup city and state info for GameTime leads. 

In [1]:
import pandas as pd
from uszipcode import SearchEngine

This is where you need to specify the name of the .csv file with the leads. I noticed that the Google sheet has pictures and some other weird stuff floating around. You'd want to delete that stuff after you download it. 

It's also possible to just pull the table right from your Google Drive, but there are some hoops to jump through. If you actually end up using any of these, we'll streamline later.

In [2]:
df = pd.read_csv('GameTime.csv')

See the file was read correctly by looking at `df`. `df` will show the entire table. `df.head()` will print the first five rows.

In [3]:
# df.head()

If it all looks good, let's keep going!

### Initial Cleanup

The original sheet is full of junk. Let's tidy up. We'll start by removing columns we don't need.

In [4]:
df=df.filter(['ad_id', 'zip_code', 'email', 'full_name'])
df.head()

,ad_id,zip_code,email,full_name
0,23844015745200267,25704,bambiicurry@gmail.com,Bambi Curry
1,23844015745200267,45030,kelishaphillips@yahoo.com,Kelisha Phillips
2,23844015745200267,29172,feb111964@gmail.com,Karen Dover
3,23844015745200267,44135,soriajoshua79@gmail.com,Joshua Soria
4,23844015745180267,37381,livin4love6932@gmail.com,Nikki Roark Lewis


We'll use this little lookup table to give our ads meaningful names.

In [5]:
ads = {23844015745170267:'playgrounds',
       23844015745180267:'thrive',
       23844015745200267:'catalogs',
   }

These two lines will a) associate an `ad_id` with a name and b) remove the `ad_id` column.

In [6]:
df['ad_name'] = df['ad_id'].map(ads)
df=df.filter(['zip_code', 'email', 'full_name', 'ad_name'])
df.head()

,zip_code,email,full_name,ad_name
0,25704,bambiicurry@gmail.com,Bambi Curry,catalogs
1,45030,kelishaphillips@yahoo.com,Kelisha Phillips,catalogs
2,29172,feb111964@gmail.com,Karen Dover,catalogs
3,44135,soriajoshua79@gmail.com,Joshua Soria,catalogs
4,37381,livin4love6932@gmail.com,Nikki Roark Lewis,thrive


The next bit just sets us up to use the uszipcode library. See docs for [help](https://uszipcode.readthedocs.io).<br><br>
I'm showing the result of `search.by_zipcode("37415")` so you can see what data is available.

In [7]:
search = SearchEngine(simple_zipcode=True)  # set simple_zipcode=False to use rich info database
# zipcode = search.by_zipcode("37415")
# zipcode

So first,  let's pull the post office city. <br><br>
This command creates a new row called `post_office_city` then uses the value in `zip_code` do lookup the post office city. 

In [8]:
df['post_office_city'] = df['zip_code'].apply(lambda x: search.by_zipcode(x).post_office_city)
df.head()

,zip_code,email,full_name,ad_name,post_office_city
0,25704,bambiicurry@gmail.com,Bambi Curry,catalogs,"Huntington, WV"
1,45030,kelishaphillips@yahoo.com,Kelisha Phillips,catalogs,"Harrison, OH"
2,29172,feb111964@gmail.com,Karen Dover,catalogs,"West Columbia, SC"
3,44135,soriajoshua79@gmail.com,Joshua Soria,catalogs,"Cleveland, OH"
4,37381,livin4love6932@gmail.com,Nikki Roark Lewis,thrive,"Spring City, TN"


If you want more, just add 'em in.

In [9]:
df['city'] = df['zip_code'].apply(lambda x: search.by_zipcode(x).major_city)
df['state'] = df['zip_code'].apply(lambda x: search.by_zipcode(x).state)
# df['county'] = df['zip_code'].apply(lambda x: search.by_zipcode(x).county)

df.head()

,zip_code,email,full_name,ad_name,post_office_city,city,state
0,25704,bambiicurry@gmail.com,Bambi Curry,catalogs,"Huntington, WV",Huntington,WV
1,45030,kelishaphillips@yahoo.com,Kelisha Phillips,catalogs,"Harrison, OH",Harrison,OH
2,29172,feb111964@gmail.com,Karen Dover,catalogs,"West Columbia, SC",West Columbia,SC
3,44135,soriajoshua79@gmail.com,Joshua Soria,catalogs,"Cleveland, OH",Cleveland,OH
4,37381,livin4love6932@gmail.com,Nikki Roark Lewis,thrive,"Spring City, TN",Spring City,TN


### Capitalization corrections

You might also want to run over the names to make sure the caps are set the way you want them. This command uses `title` whcih returns a title cased version of the string, which means the first character of the each word is capitalized.

In [10]:
df.full_name.str.title()
df.head()

,zip_code,email,full_name,ad_name,post_office_city,city,state
0,25704,bambiicurry@gmail.com,Bambi Curry,catalogs,"Huntington, WV",Huntington,WV
1,45030,kelishaphillips@yahoo.com,Kelisha Phillips,catalogs,"Harrison, OH",Harrison,OH
2,29172,feb111964@gmail.com,Karen Dover,catalogs,"West Columbia, SC",West Columbia,SC
3,44135,soriajoshua79@gmail.com,Joshua Soria,catalogs,"Cleveland, OH",Cleveland,OH
4,37381,livin4love6932@gmail.com,Nikki Roark Lewis,thrive,"Spring City, TN",Spring City,TN


### Column ordering

And maybe you want the columns reordered.

In [11]:
df = df[['full_name','email','post_office_city','city','state', 'zip_code', 'ad_name']]
df.head()

,full_name,email,post_office_city,city,state,zip_code,ad_name
0,Bambi Curry,bambiicurry@gmail.com,"Huntington, WV",Huntington,WV,25704,catalogs
1,Kelisha Phillips,kelishaphillips@yahoo.com,"Harrison, OH",Harrison,OH,45030,catalogs
2,Karen Dover,feb111964@gmail.com,"West Columbia, SC",West Columbia,SC,29172,catalogs
3,Joshua Soria,soriajoshua79@gmail.com,"Cleveland, OH",Cleveland,OH,44135,catalogs
4,Nikki Roark Lewis,livin4love6932@gmail.com,"Spring City, TN",Spring City,TN,37381,thrive


### When you're done, export it!

Finally, let's export it out as a new .csv file.

In [12]:
df.to_csv('leads.csv', index=False)

In [13]:
df['ad_name'].value_counts()

catalogs       43
thrive          6
playgrounds     3
Name: ad_name, dtype: int64

# Testing

In [14]:
# %matplotlib inline

# import matplotlib.pyplot as plt
# import seaborn as sns

# ad_counts  = df['ad_name'].value_counts()
# plt.figure(figsize=(10,5))
# sns.barplot(ad_counts.index, ad_counts.values, alpha=0.8)
# plt.title('Ad results')
# plt.ylabel('Number of Occurrences', fontsize=12)
# plt.xlabel('Ad Name', fontsize=12)
# plt.show()